In [2]:
import csv, nltk, pickle, re, time
import pandas as pd
import numpy as np
from io import StringIO
from collections import Counter
from scipy import sparse
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

# need to use once to download nltk (natural language processing library) on your computer.
# nltk.download()

In [3]:
# This function extracts tweet data and puts it in a dataframe
def get_tweets(filename):
    
    # Read the data file
    with open("twitter-datasets/" + filename, "r", encoding="utf8") as myfile:
        data = myfile.readlines()
        
    # Make a dataframe out of the data
    return pd.DataFrame(data)

In [14]:
neg_DF = get_tweets("train_neg_proc.txt")
pos_DF = get_tweets("train_pos_proc.txt")
neg_DF

,0
0,vinco tresorpack difficulty object disassemble...
1,glad dot taks tomorrow thankful startho\n
2,v celtic regular season fucked play playoff\n
3,could actually kill girl sorry\n
4,find hard believe im afraid\n
5,wish could night tonight\n
6,got kicked wgm\n
7,yes tell lip closed okay\n
8,perfect\n
9,hi harry havea good time au didnt get see mayb...


In [8]:
# create the vectoriser
neg_vectorizer = TfidfVectorizer()
pos_vectorizer = TfidfVectorizer()

In [12]:
# put the list of words into a usable format
neg_DF = pd.DataFrame(neg_DF)
pos_DF = pd.DataFrame(pos_DF)
neg_DF[0] = neg_DF[0].apply(' '.join)
pos_DF[0] = pos_DF[0].apply(' '.join)

In [13]:
neg_DF

,0
0,v i n c o t r e s o r ...
1,g l a d d o t t a k ...
2,v c e l t i c r e g ...
3,c o u l d a c t u a l ...
4,f i n d h a r d b e ...
5,w i s h c o u l d n ...
6,g o t k i c k e d w ...
7,y e s t e l l l i p ...
8,p e r f e c t \n
9,h i h a r r y h a v ...


In [15]:
# train each vectorizer on the respective corpus
x_neg = neg_vectorizer.fit_transform(neg_DF[0])
x_pos = pos_vectorizer.fit_transform(pos_DF[0])

In [19]:
n_components = 10

lda_neg = LatentDirichletAllocation(max_iter=5, 
                                learning_method='online', learning_offset=50., random_state=0)
lda_pos = LatentDirichletAllocation(max_iter=5, 
                                learning_method='online', learning_offset=50., random_state=0)

In [ ]:
lda_neg.fit(x_neg)
lda_pos.fit(x_pos)

In [ ]:
# function given by tfidf tutorial on sklearn
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
n_top_words = 20

tf_neg_features = neg_vectorizer.get_feature_names()
print_top_words(lda_neg, tf_neg_features, n_top_words)

In [ ]:
tf_pos_features = pos_vectorizer.get_feature_names()
print_top_words(lda_pos, tf_pos_features, n_top_words)

Je suis pas sûre que ce soit hyper utile ou de ce qu'il faille faire avec le TF-IDF en faite.